# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

## Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [4]:
outage_df = pd.read_excel('outage.xlsx')
# outage_df.loc[4:]
test_df = pd.DataFrame(columns=outage_df.iloc[4].tolist()[1:])
test_df

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND


In [5]:
#CLEANING
a = dict(zip(list(pd.DataFrame(outage_df.iloc[5:, 1:]).columns), outage_df.iloc[4].tolist()[1:]))
outage_cleaned = pd.DataFrame(outage_df.iloc[6:, 1:]).rename(columns=a).reset_index().drop(['index'], axis='columns')
outage_cleaned['OUTAGE.START'] = (outage_cleaned['OUTAGE.START.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.START.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned['OUTAGE.RESTORATION'] = (outage_cleaned['OUTAGE.RESTORATION.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.RESTORATION.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis='columns')

In [ ]:
#MAR test
#take states
#make columns missing=True and missing=False for customers_affected missing or not
#groupby, take counts of true/false columns fuck pivot table
mar_test = outage_cleaned.copy()
mar_test['missingTrue'] = mar_test['CUSTOMERS.AFFECTED'].isna()
mar_test['missingFalse'] = mar_test['CUSTOMERS.AFFECTED'].notna()
observed_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2

tvds = []
for _ in range(100):
    mar_test['U.S._STATE'] = np.random.permutation(mar_test['U.S._STATE'])
    perm_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
    tvds.append(perm_tvd)
np.mean(tvds > observed_tvd)
# tvds

In [ ]:
#question: does month have an effect on the severity of power outages?
#null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.
#alternate hypothesis: the summer months have greater power outages than rest of the year
#note: use mean to do permutation test
#accounting for outliers of both sets, but outliers are often urban centers which are important

#find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months
# shuffle month column 1000 times, find diff, find percentage of those that had higher diff than observed

observed_diff = outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_cleaned['CUSTOMERS.AFFECTED'].mean()

outage_copy = outage_cleaned.copy()
p_vals = []
diffs_list = []
for __ in range(100):
    for _ in range(1000):
        outage_copy['MONTH'] = np.random.permutation(outage_copy['MONTH'])
        perm_diff = outage_copy[(outage_copy['MONTH'] == 7) | (outage_copy['MONTH'] == 8) | (outage_copy['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_copy['CUSTOMERS.AFFECTED'].mean()
        diffs_list.append(perm_diff)
    p_vals.append(np.mean(diffs_list > observed_diff))
np.mean(p_vals)

In [61]:
a = dict(zip(list(pd.DataFrame(outage_df.iloc[5:, 1:]).columns), outage_df.iloc[4].tolist()[1:]))
outage_cleaned = pd.DataFrame(outage_df.iloc[6:, 1:]).rename(columns=a).reset_index().drop(['index'], axis='columns')
outage_cleaned[outage_cleaned.isna().any(axis=1)]
outage_cleaned.T.isna().sum(axis=1).sort_values(ascending=False)


HURRICANE.NAMES            1462
DEMAND.LOSS.MW              705
CAUSE.CATEGORY.DETAIL       471
CUSTOMERS.AFFECTED          443
OUTAGE.DURATION              58
OUTAGE.RESTORATION.TIME      58
OUTAGE.RESTORATION.DATE      58
IND.PRICE                    22
COM.PRICE                    22
COM.PERCEN                   22
RES.SALES                    22
COM.SALES                    22
IND.SALES                    22
TOTAL.SALES                  22
RES.PERCEN                   22
IND.PERCEN                   22
TOTAL.PRICE                  22
RES.PRICE                    22
POPDEN_RURAL                 10
POPDEN_UC                    10
OUTAGE.START.TIME             9
OUTAGE.START.DATE             9
CLIMATE.CATEGORY              9
ANOMALY.LEVEL                 9
MONTH                         9
CLIMATE.REGION                6
POPPCT_UC                     0
TOTAL.REALGSP                 0
UTIL.CONTRI                   0
PI.UTIL.OFUSA                 0
POPULATION                    0
POPPCT_U

In [68]:
outage_cleaned[outage_cleaned['IND.PRICE'].isna()]

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
36,37,2016,7,Tennessee,TN,SERC,Central,-0.3,normal,2016-07-13 00:00:00,15:00:00,NaN,NaN,system operability disruption,public appeal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2812287,480032,1193,3293512,85.388698,14.57508,0.036223,43720,50660,0.863008,3,1813,290712,0.623641,0.5,6649404,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.843109,2.156891,2.156891
103,104,2016,7,Michigan,MI,RFC,East North Central,-0.3,normal,2016-07-08 00:00:00,19:00:00,2016-07-09 00:00:00,00:00:00,severe weather,NaN,NaN,300,NaN,160895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4311008,538677,6191,4855877,88.779184,11.0933,0.127495,43330,50660,0.85531,2.6,8303,430412,1.929082,3.8,9933445,74.57,8.19,2034.1,1390.4,47.5,6.41,1.03,58.459995,41.540005,2.068987
171,172,2016,7,Texas,TX,TRE,South,-0.3,normal,2016-07-05 00:00:00,02:45:00,2016-07-06 00:00:00,03:00:00,severe weather,NaN,NaN,1455,NaN,52000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10521732,1447247,105552,12074534,87.13986,11.985945,0.87417,53104,50660,1.048243,-1.1,31136,1481866,2.101135,8.5,27904862,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873
218,219,2016,7,Texas,TX,TRE,South,-0.3,normal,2016-07-09 00:00:00,17:45:00,2016-07-11 00:00:00,14:00:00,severe weather,NaN,NaN,2655,NaN,62000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10521732,1447247,105552,12074534,87.13986,11.985945,0.87417,53104,50660,1.048243,-1.1,31136,1481866,2.101135,8.5,27904862,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873
239,240,2000,NaN,Texas,TX,FRCC,South,NaN,NaN,NaN,NaN,NaN,NaN,equipment failure,transformer outage,NaN,NaN,46,43000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8023266,1093414,61280,9299829,86.273264,11.757356,0.658937,45102,44745,1.007979,1.7,30908,944631,3.271965,10.3,20944499,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873
339,340,2000,NaN,Alabama,AL,SERC,Southeast,NaN,NaN,NaN,NaN,NaN,NaN,severe weather,thunderstorm,NaN,NaN,NaN,160000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1930037,313017,6252,2262753,85.295965,13.833459,0.276301,33712,44745,0.753425,1.1,5704,150090,3.800386,2.2,4452173,59.04,10.39,1278.5,988.7,40.4,4.36,0.99,96.613888,3.386112,2.018314
365,366,2000,NaN,Illinois,IL,SERC,Central,NaN,NaN,NaN,NaN,NaN,NaN,severe weather,wildfire,NaN,NaN,NaN,11000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4748863,493670,5009,5282401,89.899707,9.345561,0.094824,49276,44745,1.101263,2.8,15815,612709,2.58116,5.8,12434161,88.49,8.52,2877.6,1759.5,28.6,7.11,1.12,95.864558,4.135442,1.415893
417,418,2016,7,Washington,WA,WECC,Northwest,-0.3,normal,2016-07-13 00:00:00,13:00:00,2016-07-13 00:00:00,13:01:00,intentional attack,vandalism,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2985799,367847,29012,3382664,88.267679,10.874476,0.857667,57796,50660,1.140861,4.3,3504,420809,0.832682,0.7,7280934,84.05,9.08,2380,1487.9,16.7,3.57,0.62,93.208786,6.791214,2.405397
605,606,2016,7,Pennsylvania,PA,RFC,Northeast,-0.3,normal,2016-07-23 00:00:00,15:15:00,2016-07-23 00:00:00,19:53:00,system operability disruption,voltage reduction,NaN,278,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5335555,696080,23372,6055013,88.117978,11.495929,0.385994,50978,50660,1.006277,4.2,10659,651857,1.635175,4.1,12787085,78

In [132]:
# outage_cleaned['OUTAGE.START.TIME'][0] + outage_cleaned['OUTAGE.START.DATE'][0]
# outage_cleaned['OUTAGE.START.DATE'][0].date()
# outage_cleaned['OUTAGE.START.DATE'].transform(lambda x: x.replace(hour=1, minute=10))
# outage_cleaned['OUTAGE.START.TIME'][0].hour

test_copy = outage_cleaned.copy()
# test_copy['OUTAGE.START.DATE'].dt.date + pd.to_timedelta(test_copy['OUTAGE.START.TIME'].dt.strftime('%H:%M:%S'))
test_copy['OUTAGE.START.TIME'][0].hour
# new_date = pd.datetime(test_copy['OUTAGE.START.DATE'][0].year, test_copy['OUTAGE.START.DATE'][0].month, test_copy['OUTAGE.START.DATE'][0].day, test_copy['OUTAGE.START.TIME'][0].hour, test_copy['OUTAGE.START.TIME'][0].minute, 0)

# test_copy['test_date'] = test_copy.apply(lambda x, y: pd.datetime(x.year, x.month, x.date, y.hour, y.minute, 0), test_copy['OUTAGE.START.DATE'], test_copy['OUTAGE.START.TIME'])
# pd.to_datetime(test_copy['OUTAGE.START.TIME'][0])
# datetime_object = datetime.combine(datetime.date.today(), time)

pd.datetime.combine(pd.datetime.today().date(), test_copy['OUTAGE.START.TIME'][0])
test_copy['OUTAGE.START.TIME'] = test_copy['OUTAGE.START.TIME'].apply(lambda x: pd.datetime.combine(pd.datetime.today().date(), x) if type(x) != float else pd.datetime(2023, 1, 1, 0, 0, 0))
test_copy['OUTAGE.START.TIME'][0]
test_copy['OUTAGE.START.DATE'].dt.date + pd.to_timedelta(test_copy['OUTAGE.START.TIME'])#.dt.strftime('%H:%M:%S'))


/var/folders/75/4b4y2mlx2yv_34rkxb83ydkr0000gn/T/ipykernel_91353/1668987297.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime.combine(pd.datetime.today().date(), test_copy['OUTAGE.START.TIME'][0])
/var/folders/75/4b4y2mlx2yv_34rkxb83ydkr0000gn/T/ipykernel_91353/1668987297.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  test_copy['OUTAGE.START.TIME'] = test_copy['OUTAGE.START.TIME'].apply(lambda x: pd.datetime.combine(pd.datetime.today().date(), x) if type(x) != float else pd.datetime(2023, 1, 1, 0, 0, 0))


AttributeError: Can only use .dt accessor with datetimelike values

In [209]:
test_copy = outage_cleaned.copy()
# test_copy['OUTAGE.START.DATE'][0].hour = 1
# test_copy['test_date'] = test_copy.apply(lambda x: pd.datetime(x.loc['OUTAGE.START.DATE'].year, x.loc['OUTAGE.START.DATE'].month, x.loc['OUTAGE.START.DATE'].date, x.loc['OUTAGE.START.TIME'].hour, x.loc['OUTAGE.START.TIME'].minute, 0))
# pd.datetime.combine(test_copy['OUTAGE.START.DATE'][0].date(), test_copy['OUTAGE.START.TIME'][0])
# test_copy['test_date'] = test_copy.apply(lambda x: True if x['U.S._STATE']=='Minnesota' else False)
# test_copy
# test_copy['OUTAGE.START.DATE'][0] = pd.to_datetime(str(test_copy['OUTAGE.START.DATE'][0]).split(' ')[0] + ' ' + str(test_copy['OUTAGE.START.TIME'][0]))

test_copy['OUTAGE.START'] = (test_copy['OUTAGE.START.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  test_copy['OUTAGE.START.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
test_copy['OUTAGE.RESTORATION'] = (test_copy['OUTAGE.RESTORATION.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  test_copy['OUTAGE.RESTORATION.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
test_copy = test_copy.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis='columns')

In [208]:
# type(test_copy.iloc[-1]['OUTAGE.START.DATE'])
test_copy.tail(5)

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
1529,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,NaN,NaN,720,155,34500,8.41,7.8,6.2,7.56,488853,438133,386693,1313678,37.212544,33.351628,29.435904,330738,60017,3639,394394,83.859795,15.217524,0.922681,57012,47586,1.198083,9.8,934,39067,2.390765,0.5,685326,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1530,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,1650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309997,53709,2331,366037,84.690072,14.673107,0.636821,42913,48909,0.877405,3.5,1019,27868,3.656524,0.7,649422,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,NaT,NaT
1531,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,NaN,NaN,59,84,NaN,9.25,7.47,5.53,7.67,337874,370771,215406,924051,36.564432,40.124517,23.311051,367206,65971,3052,436229,84.177347,15.12302,0.699633,45230,46680,0.968937,0,606,36504,1.660092,0.3,807067,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00
1532,1533,2009,8,South Dakota,SD,MRO,West North Central,0.5,warm,islanding,NaN,NaN,181,373,NaN,9.25,7.47,5.53,7.67,337874,370771,215406,924051,36.564432,40.124517,23.311051,367206,65971,3052,436229,84.177347,15.12302,0.699633,45230,46680,0.968937,0,606,36504,1.660092,0.3,807067,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256,2009-08-29 11:00:00,2009-08-29 14:01:00
1533,1534,2000,NaN,Alaska,AK,ASCC,NaN,NaN,NaN,equipment failure,failure,NaN,NaN,35,14273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230534,38074,854,273530,84.281066,13.919497,0.312214,57401,44745,1.282847,-2.2,724,36046,2.008545,0.2,627963,66.02,21.56,1802.6,1276,0.4,0.05,0.02,85.761154,14.238846,2.901182,NaT,NaT


In [96]:
outage_cleaned

OBS                                          1
YEAR                                      2011
MONTH                                        7
U.S._STATE                           Minnesota
POSTAL.CODE                                 MN
NERC.REGION                                MRO
CLIMATE.REGION              East North Central
ANOMALY.LEVEL                             -0.3
CLIMATE.CATEGORY                        normal
OUTAGE.START.DATE          2011-07-01 00:00:00
OUTAGE.START.TIME                     17:00:00
OUTAGE.RESTORATION.DATE    2011-07-03 00:00:00
OUTAGE.RESTORATION.TIME               20:00:00
CAUSE.CATEGORY                  severe weather
CAUSE.CATEGORY.DETAIL                      NaN
HURRICANE.NAMES                            NaN
OUTAGE.DURATION                           3060
DEMAND.LOSS.MW                             NaN
CUSTOMERS.AFFECTED                       70000
RES.PRICE                                 11.6
COM.PRICE                                 9.18
IND.PRICE    

In [80]:
pd.set_option('max_column', None)
# outage_cleaned[outage_cleaned['IND.PRICE'].isna()]
# outage_cleaned[outage_cleaned['OUTAGE.DURATION'].isna()]

In [ ]:
# MAR: state vs customers missing
# NMAR: demand loss, cause category detail
#just do cause category, missing because cannot properly fit into 

In [228]:
#MAR test
#take states
#make columns missing=True and missing=False for customers_affected missing or not
#groupby, take counts of true/false columns fuck pivot table
mar_test = outage_cleaned.copy()
mar_test['missingTrue'] = mar_test['CUSTOMERS.AFFECTED'].isna()
mar_test['missingFalse'] = mar_test['CUSTOMERS.AFFECTED'].notna()
observed_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2

tvds = []
for _ in range(100):
    mar_test['U.S._STATE'] = np.random.permutation(mar_test['U.S._STATE'])
    perm_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
    tvds.append(perm_tvd)
np.mean(tvds > observed_tvd)
# tvds

/var/folders/75/4b4y2mlx2yv_34rkxb83ydkr0000gn/T/ipykernel_91353/2523239759.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  observed_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
/var/folders/75/4b4y2mlx2yv_34rkxb83ydkr0000gn/T/ipykernel_91353/2523239759.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  perm_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2


0.0

In [255]:
#question: does month have an effect on the severity of power outages?
#null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.
#alternate hypothesis: the summer months have greater power outages than rest of the year
#note: use mean to do permutation test
#accounting for outliers of both sets, but outliers are often urban centers which are important

#find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months
# shuffle month column 1000 times, find diff, find percentage of those that had higher diff than observed
outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median()

80000.0

In [252]:
observed_diff = outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median() - outage_cleaned['CUSTOMERS.AFFECTED'].median()

outage_copy = outage_cleaned.copy()
p_vals = []
diffs_list = []
for __ in range(100):
    for _ in range(1000):
        outage_copy['MONTH'] = np.random.permutation(outage_copy['MONTH'])
        perm_diff = outage_copy[(outage_copy['MONTH'] == 7) | (outage_copy['MONTH'] == 8) | (outage_copy['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median() - outage_copy['CUSTOMERS.AFFECTED'].median()
        diffs_list.append(perm_diff)
    p_vals.append(np.mean(diffs_list > observed_diff))
np.mean(p_vals)

0.041816459248813144

In [256]:
#question: does month have an effect on the severity of power outages?
#null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.
#alternate hypothesis: the summer months have greater power outages than rest of the year
#note: use mean to do permutation test
#accounting for outliers of both sets, but outliers are often urban centers which are important

#find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months
# shuffle month column 1000 times, find diff, find percentage of those that had higher diff than observed

observed_diff = outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_cleaned['CUSTOMERS.AFFECTED'].mean()

outage_copy = outage_cleaned.copy()
p_vals = []
diffs_list = []
for __ in range(100):
    for _ in range(1000):
        outage_copy['MONTH'] = np.random.permutation(outage_copy['MONTH'])
        perm_diff = outage_copy[(outage_copy['MONTH'] == 7) | (outage_copy['MONTH'] == 8) | (outage_copy['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_copy['CUSTOMERS.AFFECTED'].mean()
        diffs_list.append(perm_diff)
    p_vals.append(np.mean(diffs_list > observed_diff))
np.mean(p_vals)

0.008334130453516959

### Cleaning and EDA

In [ ]:
# TODO

### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Testing

In [ ]:
# the time of the year has no effect on the severity of power outages (how many customers affected)